In [65]:
import pandas as pd
import json
import requests
from tqdm import tqdm
from datetime import datetime
df = pd.read_excel('20221011_upload_pessoa_not_mapped.xlsx',sheet_name="Import", converters={'PHONE_IBD':str})
# Convert Phone number to valid value 
df['PHONE_IBD'] = df['PHONE_IBD'].str.replace('\+55','55-',n=2,regex=True)
#print(df['PHONE_IBD'])
!python -V

Python 3.9.12


In [66]:
#df['externalId'] = ""  # if no id provided  API will generate
df['marketingClientId']=None  # for confirmation
df['parentClientId']=None     # for confirmation
df['password']=None           # for confirmation
df['industry']="Other"        # for confirmation
#df['accountManager']=""       # for confirmation
df['accountStatus']="Active"
df['businessUnit']="IBD"  # for confirmation
df['leadType']="New"      # for confirmation
df['inspectionPotentialTO']="200"  # for confirmation
df['auditPotentialTO']="100"    # for confirmation
df['labTestPotentialTO']="400"  # for confirmation
df['saberPotentialTO']="300"    # for confirmation
df['comments']=None    # for confirmation
df['altEmail']=None    # for confirmation
df['accountingEmail']=None
df['salutation']=None    # for confirmation
df['jobTitle']="Manager"      # for confirmation (Mandatory)  use Manager by default
df['keyAccount']=False   # for confirmation
df['unitNumber']=None    # for confirmation
df['municipality']=None  # for confirmation
df['districtCode']=None  # for confirmation
df['streetType']=None    # for confirmation
df['acquisition']=None   # for confirmation

#streetName = STREET_IBD  available value is number


renamed_df = df.rename(columns = {"EXTERNAL_COMPANY_ID_IBD":"externalId"
                                  ,"FIRSTNAME_IBD":"firstName"
                                  ,"LASTNAME_IBD":"lastName"
                                  ,"EMAIL":"email"
                                  ,"COMPANYNAME_IBD":"companyName"
                                  ,"PHONE_IBD":"phoneNumber"
                                  ,"SIC_NAME":"accountManager"
                                  ,"COUNTRY_IBD":"country"
                                  ,"ADDRESS_IBD":"address"
                                  ,"ZIP_IBD":"postCode"
                                  ,"CURRENCY_IBD":"invoiceCurrency"
                                  ,"AIBANK_IBD":"aiBank"
                                  ,"LOGIN":"login"
                                  ,"STATE_CODE":"state"
                                  ,"CITY_IBD":"city"
                                  ,"CITY_CODE":"cityCode"
                                  ,"STATE_NAME":"stateInLocal"
                                  ,"COMPANY_SHORT_NAME_IBD":"companyNameInLocal"
                                  ,"HOUSE_NUMBER_IBD":"houseNumber"
                                  ,"DISTRICT_IBD":"district"
                                  ,"STREET_IBD":"streetName"
                                  ,"TAX_SITUATION_IBD":"TAX_SITUATION"
                                  ,"TAX_NUMBER_IBD":"VAT_NUMBER"
                                  ,"STATE_TAX_ID_IBD":"STATE_TAX_ID"
                                  })

arranged_renamed_df = renamed_df[['externalId'
                                  ,'companyName'
                                  ,'marketingClientId'
                                  ,'parentClientId'
                                  ,'login'
                                  ,'password'
                                  ,'industry'
                                  ,'accountManager'
                                  ,'accountStatus'
                                  ,'businessUnit'
                                  ,'leadType'
                                  ,'inspectionPotentialTO'
                                  ,'auditPotentialTO'
                                  ,'labTestPotentialTO'
                                  ,'saberPotentialTO'
                                  ,'comments'
                                  ,'country'
                                  ,'state'
                                  ,'city'
                                  ,'address'
                                  ,'email'
                                  ,'altEmail'
                                  ,'accountingEmail'
                                  ,'phoneNumber'                                 
                                  ,'postCode'
                                  ,'invoiceCurrency'
                                  ,'aiBank'
                                  ,'salutation'
                                  ,'firstName'
                                  ,'lastName'
                                  ,'jobTitle'
                                  ,'keyAccount'
                                  ,'unitNumber'
                                  ,'municipality'
                                  ,'cityCode'
                                  ,'districtCode'
                                  ,'streetType'
                                  ,'acquisition'
                                  ,'district'
                                  ,'streetName'
                                  ,'houseNumber'
                                  ,'TAX_SITUATION'
                                  ,'VAT_NUMBER'
                                  ,'STATE_TAX_ID']]
arranged_renamed_df.head(2)



,externalId,companyName,marketingClientId,parentClientId,login,password,industry,accountManager,accountStatus,businessUnit,...,cityCode,districtCode,streetType,acquisition,district,streetName,houseNumber,TAX_SITUATION,VAT_NUMBER,STATE_TAX_ID
0,26,Fazenda Amway Nutrilite do Brasil Ltda,None,None,ibd_fazenda_amway_nutrilite_do_brasil_ltda,None,Other,Priscila-M-Oliveira-Marqueti Pimentel,Active,IBD,...,1461.0,None,None,None,Zona Rural,26,S/N,CNPJ,02.038.405.0001-44,06.668.572-8
1,89,Companhia Cacique de Café Solúvel,None,None,ibd_companhia_cacique_de_cafe_soluvel,None,Other,Adriane-Cristina Ribeiro,Active,IBD,...,3149.0,None,None,None,Pq Indl Cacique,26,100,CNPJ,78.588.415/0001-15,60.102.504-37


In [67]:
## Script to check already exixt record based on login, and company_name ( external_id)
#SELECT  cc.id company_id, cc.company_name, ccu.user_id, gu.first_name||' '||gu.last_name company_user, gu.personal_email,  gu.login company_login, cem.id as main_external_id, cem.external_system, cem.external_id, cem.qima_client_id
#FROM AIDATAUSER.CRM_COMPANY_USER ccu  
#   inner join AIDATAUSER.CRM_COMPANY cc on ccu.company_id = cc.id
#   left join AIDATAUSER.GENERAL_USER gu on ccu.user_id = gu.id
#   left join AIDATAUSER.CLIENT_EXTERNAL_MAPPING cem on cc.id = cem.qima_client_id
# -- where cc.id in ('669BD08995754C8B82AF20ED4CEE12FB')   
# -- where ccu.user_id in ('FFE1949B608846A6B361F7A848216137')
#  where gu.login = 'ibd_antonio_carlos_da_costa_sitio_sao_sebastiao'
# -- where cc.company_name like '%Tanac%'
# --where external_id = '1143'
#################
### Check assigned SIC of Customer
# select (select key from aidatauser.parm_classified_keyvalue where id = a.status) sic_status,
#        (select key from aidatauser.parm_classified_keyvalue where id = a.sic_type) sic_type, b.company_name,
#        c.first_name||' '||c.last_name as SIC, a.* 
# from aidatauser.CRM_SALES_IN_CHARGE   a 
# left join aidatauser.crm_company b on a.company_id = b.id 
# left join aidatauser.ur_user c on a.sales_id = c.user_id
# where company_id = '04F6F3F0E5154FB298835515D3B39200'
#################
#Valid format for SIC_NAME on excel
# select FIRST_NAME || ' ' || LAST_NAME as sic_name
# from aidatauser.ur_user a where user_id in ('B04A277998B04796482587A3001B7717')
################################
#Cooperativa dos Produtores e Colhedores Orgânicos de Parnaíba -> already exist but misspelled  'Colheres' 
### Masi Agricola SPA ---> external_id 5910 existing on db (PP and PROD) and in excel Masi Agricola SPA 5873 on excel
#PP and PROD Iwao Akamatsu e outros  3603  and in excel Iwao Akamatsu e Outros (Yuri Cogumelos)  5936
#PP and PROD Cooperativa Agricola de Formosa do Rio Preto - Coafor external_id 9332 and in excel - Cooperativa Agrícola De Formosa do Rio Preto - COAFOR       6228
# PP and PROD COOPALM - Coop. Dos Prod. de Palmito do Baixo Sul da Bahia - COAFOR    internal_id   2929 and in Excel - Cooperativa dos Produtores  de Palmito do Baixo Sul da Bahia ( COOPALM)    6703
# PP and Prod Tanac S/A and in  excel Tanac S.A
# PP and Prod Associação dos Produtores de Cafés Especiais do Sul de Minas-APROCEM  and in excel Associação dos Produtores de Cafés Especiais do Sul de Minas (Aprocem)
# PP and Prod LOUISA LEE external_id 7654  and in excel Louisa Lee (Recanto Ecológico)  external_id 7658
#COMPANHIA TEXTIL DE CASTANHAL externale_id 7763 and in excel Companhia Têxtil de Castanhal external_id 7764
#OSVALDO VIU SERRANO JUNIOR external_id 734 and in excel Osvaldo Viu Serrano Junior (Sítio Fugidos) external_id 7915
# ANTONIO CARLOS DA COSTA E OUTRA external_id 3644 and in excel Antonio Carlos da Costa (Sítio São Sebastião) external_id 7916
# Perboni S/A external_id 8772 and in excel Perboni S/A external_id 8802
#RENIERIS P.C external_id 9155 and in excel Renieris P.C external_id 8939
#USINA SAO FRANCISCO S/A  external_id 169 and in excel Usina São Francisco S/A (Native) external_id 9111
#IFB - Instituto de Fosfatados Biológicos Ltda.  external_id 1143 and in excel Instituto de Fosfatos Biológicos Ltda external_id 9113
#Chemyunion Química LTDA and in excel Chemyunion Ltda_1 external_id 9181
#Mandiervas Indústria e Comércio de Ervas Medicinais Ltda external_id 3999 and in excel Mandiervas Comercio De Ervas e Condimentos Ltda external_id 9547
#ERALDO DIAS DE CASTRO JUNIOR external_id 9041 and in excel Eraldo Dias de Castro Junior external_id 9613
#Eraldo Dias de Castro Junior_9613 exclude due to duplicate record confirmed by bruno to use the correct external id 3894
company_to_exclude= [ 'Masi Agricola SPA' # exist in prod db
                      ,'Iwao Akamatsu e Outros (Yuri Cogumelos)' # exist in prod db
                      ,'Cooperativa Agrícola De Formosa do Rio Preto - COAFOR' # exist in prod db
                      ,'Cooperativa dos Produtores  de Palmito do Baixo Sul da Bahia ( COOPALM)' # exist in prod db
                      ,'Tanac S.A'  # exist in prod db
                      ,'Associação dos Produtores de Cafés Especiais do Sul de Minas (Aprocem)' # exist in prod db
                      ,'Louisa Lee (Recanto Ecológico)' # exist in prod db
                      ,'Companhia Têxtil de Castanhal'  # exist in prod db
                      ,'Osvaldo Viu Serrano Junior (Sítio Fugidos)'  # exist in prod db
                      ,'Antonio Carlos da Costa (Sítio São Sebastião)'  # exist in prod db
                      ,'Perboni S/A'  # exist in prod db
                      ,'Renieris P.C'  # exist in prod db
                      ,'Usina São Francisco S/A (Native)'  # exist in prod db
                      ,'Instituto de Fosfatos Biológicos Ltda'  # exist in prod db
                      ,'Mandiervas Comercio De Ervas e Condimentos Ltda'  # exist in prod db                    
                      ,'Eraldo Dias de Castro Junior_9613']  # exist in prod db
arranged_renamed_df = arranged_renamed_df[~arranged_renamed_df.companyName.isin(company_to_exclude)]


In [72]:
#sample_record = arranged_renamed_df.head(1)
#filter_value = ['Fazenda Amway Nutrilite do Brasil Ltda',
#'Companhia Cacique de Café Solúvel',
#'Fernando Borges de Assis',
#'NUTRITION & SANTÉ DO BRASIL LTDA',
#'Coop. dos Produtores Org. do Sul da Bahia - CABRUCA',
#'Cláudio Carneiro Pinto',
#'Marfrig Global Foods S/A']
#filter_value = ['Nuncio Regino']
#filter_value = ['Ibis Agraria Ltda']
#filter_value = ['Cumberland Packing Corp.']
filter_value = ['Biodiver do Brasil Ltda']
record = arranged_renamed_df[arranged_renamed_df['companyName'].isin(filter_value)]
#record = arranged_renamed_df
record 

,externalId,companyName,marketingClientId,parentClientId,login,password,industry,accountManager,accountStatus,businessUnit,...,cityCode,districtCode,streetType,acquisition,district,streetName,houseNumber,TAX_SITUATION,VAT_NUMBER,STATE_TAX_ID
248,9736,Biodiver do Brasil Ltda,None,None,ibd_biodiver_do_brasil_ltda_1,None,Other,Caio-Sol Bispo-dos-Santos,Active,IBD,...,4933.0,None,None,None,Vila Romana,26,324,CNPJ,43.059.474/0001-11,131.742.612.116


In [73]:
param_generateLogin = "false"
param_loginStrategy = "email"
param_generatePassword = "true"
param_passwordStrategy = "random"
param_keepOriginalPassword = "false"
param_externalSystemName = "IBD"
param_businessUnitKey = "IBD"
param_externalIdByMd5 = "false"
param_allowUpdate = "true"

In [74]:
# Assign URL
post_url_new_account = f"'http://ppservice.qima.com/customer-service/v1.0/customer?generateLogin={param_generateLogin}&loginStrategy={param_loginStrategy}&generatePassword={param_generatePassword}&passwordStrategy={param_passwordStrategy}&keepOriginalPassword={param_keepOriginalPassword}&externalSystemName={param_externalSystemName}&businessUnitKey={param_businessUnitKey}&externalIdByMd5={param_externalIdByMd5}&allowUpdate={param_allowUpdate}'"
print(post_url_new_account)

post_url_sync_netsuite="'https://ppservice.qima.com/customer-service/customer/update-invoicing-fields'"
print(post_url_sync_netsuite)



'http://ppservice.qima.com/customer-service/v1.0/customer?generateLogin=false&loginStrategy=email&generatePassword=true&passwordStrategy=random&keepOriginalPassword=false&externalSystemName=IBD&businessUnitKey=IBD&externalIdByMd5=false&allowUpdate=true'
'https://ppservice.qima.com/customer-service/customer/update-invoicing-fields'


In [75]:
v_externalId = []
v_companyName = []
v_login = []
v_city = []
v_phoneNumber = []
v_accountManager = []
v_clientId_append = []
v_clientId = []


for i in tqdm(record.index):
    if i >= 0 :
      print('Index : ',i)  
      data = json.dumps(json.loads(record.loc[i].to_json(orient='index')), indent=4)   
      data = json.loads(data)
      post_new_account =eval(post_url_new_account)
      response = requests.post(post_new_account,json=data)
      print(data)
#      #print ('Status_code = ',response.status_code,'response content = ',response.content)   
#      if response.status_code == 200:  ## 200 OK	Success on GET, PUT, or POST. Returned for a successful response
#          get_customerId = json.loads(response.content)
#          customerId = get_customerId['content']['clientId']        
#          v_clientId_append.append(customerId)
#          if(customerId and not customerId.isspace()):
#              #print("The string is not empty",customerId,' Login ', [record.loc[i]['login']], ' Company Name ', [record.loc[i]['companyName']])
#              #if data['VAT_NUMBER'] is None:                  
#              #    if data['STATE_TAX_ID'] is None:
#              #         data_sync_netsuite = [{
#              #              "customerId":  customerId
#              #              ,"TAX_SITUATION": data['TAX_SITUATION']
#              #              ,"STATE_TAX_ID":data['STATE_TAX_ID']
#              #              ,"VAT_NUMBER": data['VAT_NUMBER']
#              #              ,"invoiceCurrency": data['invoiceCurrency']
#              #              ,"aiBank":data['aiBank']  
#              #              }]
#              #    else:
#              #        data_sync_netsuite = [{
#              #              "customerId":  customerId
#              #              ,"TAX_SITUATION": data['TAX_SITUATION']
#              #              ,"STATE_TAX_ID":data['STATE_TAX_ID']
#              #              ,"VAT_NUMBER": data['VAT_NUMBER']
#              #              ,"invoiceCurrency": data['invoiceCurrency']
#              #              ,"aiBank":data['aiBank']  
#              #              }]
#              #else:
#              #    if data['STATE_TAX_ID'] is None:
#              #        data_sync_netsuite = [{
#              #              "customerId":  customerId
#              #              ,"TAX_SITUATION": data['TAX_SITUATION']
#              #              ,"STATE_TAX_ID":data['STATE_TAX_ID']
#              #              ,"VAT_NUMBER": data['VAT_NUMBER']
#              #              ,"invoiceCurrency": data['invoiceCurrency']
#              #              ,"aiBank":data['aiBank']  
#              #              }]
#              #    else: 
#              #        data_sync_netsuite = [{
#              #              "customerId":  customerId
#              #              ,"TAX_SITUATION": data['TAX_SITUATION']   ## Mandatory field in QSP
#              #              ,"STATE_TAX_ID":data['STATE_TAX_ID']
#              #              ,"VAT_NUMBER": data['VAT_NUMBER']
#              #              ,"invoiceCurrency": data['invoiceCurrency']
#              #              ,"aiBank":data['aiBank']  
#              #              }] 
#              data_sync_netsuite = [{
#                            "customerId":  customerId
#                            ,"TAX_SITUATION": data['TAX_SITUATION']   ## Mandatory field in QSP
#                            ,"STATE_TAX_ID":data['STATE_TAX_ID']
#                            ,"VAT_NUMBER": data['VAT_NUMBER']
#                            ,"invoiceCurrency": data['invoiceCurrency']
#                            ,"aiBank":data['aiBank']  
#                            }]           
#              post_sync_netsuite =eval(post_url_sync_netsuite)
#              response_sync_netsuite = requests.post(post_sync_netsuite,json=data_sync_netsuite)
#              if response_sync_netsuite.status_code == 200:
#                  v_clientId = v_clientId_append
#                  v_externalId += [record.loc[i]['externalId']]
#                  v_companyName += [record.loc[i]['companyName']]
#                  v_login += [record.loc[i]['login']]
#                  v_city += [record.loc[i]['city']]
#                  v_phoneNumber += [record.loc[i]['phoneNumber']]
#                  v_accountManager += [record.loc[i]['accountManager']]
#              else:
#                  response.status_code = 400
#                      
#              
#      if response.status_code == 400:
#          print ( 'Customer Status_code= ',response.status_code,
#                  'Customer response content = ',response.content,
#                  'Netsuite Status_code = ',response_sync_netsuite.status_code,
#                  'Netusite response content = ',response_sync_netsuite.content                  
#                )
#          break
#      else:
#          print ( 'Customer Status_code= ',response.status_code,
#                  'Customer response content = ',response.content,
#                  'Netsuite Status_code = ',response_sync_netsuite.status_code,
#                  'Netusite response content = ',response_sync_netsuite.content
#                  )
#                  
#df_create = pd.DataFrame({
#    'clientId':v_clientId 
#    ,'externalId':v_externalId 
#    ,'companyName':v_companyName 
#    ,'login':v_login
#    ,'city':v_city 
#    ,'phoneNumber':v_phoneNumber     
#    ,'accountManager':v_accountManager
#                         })

today = datetime.now()
writer = pd.ExcelWriter(f'output/recorded_clients{today.year}_{today.month}_{today.day}_{today.hour}_{today.minute}_{today.second}.xlsx')
df_create.to_excel(writer, sheet_name='Sheet1')  
writer.save()

  0%|          | 0/1 [00:00<?, ?it/s]

Index :  248


100%|██████████| 1/1 [00:01<00:00,  1.42s/it]

{'externalId': 9736, 'companyName': 'Biodiver do Brasil Ltda', 'marketingClientId': None, 'parentClientId': None, 'login': 'ibd_biodiver_do_brasil_ltda_1', 'password': None, 'industry': 'Other', 'accountManager': 'Caio-Sol Bispo-dos-Santos', 'accountStatus': 'Active', 'businessUnit': 'IBD', 'leadType': 'New', 'inspectionPotentialTO': '200', 'auditPotentialTO': '100', 'labTestPotentialTO': '400', 'saberPotentialTO': '300', 'comments': None, 'country': 'Brazil', 'state': 'SP', 'city': 'São Paulo - SP', 'address': 'Rua Mipibu, 324 - Vila Romana', 'email': 'rafael@biodiver.com.br', 'altEmail': None, 'accountingEmail': None, 'phoneNumber': '0000000000', 'postCode': '05049030', 'invoiceCurrency': 'BRL', 'aiBank': 49, 'salutation': None, 'firstName': 'RAFAEL', 'lastName': 'AVEZUM DE ALMEIDA', 'jobTitle': 'Manager', 'keyAccount': False, 'unitNumber': None, 'municipality': None, 'cityCode': 4933.0, 'districtCode': None, 'streetType': None, 'acquisition': None, 'district': 'Vila Romana', 'street